### 基础操作

In [1]:
import json

# Python对象 → JSON字符串
data = {"name": "张三", "age": 25, "city": "北京"}
json_str = json.dumps(data)
print(json_str)  
# {"name": "\u5f20\u4e09", "age": 25, "city": "\u5317\u4eac"}

# JSON字符串 → Python对象
data2 = json.loads(json_str)
print(data2["name"])  # 张三

{"name": "\u5f20\u4e09", "age": 25, "city": "\u5317\u4eac"}
张三


### 文件读写

In [ ]:
import json

# 写入JSON文件
data = {"name": "李四", "scores": [85, 92, 78]}
with open("data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

# 读取JSON文件
with open("data.json", "r", encoding="utf-8") as f:
    loaded = json.load(f)
    print(loaded["name"])  # 李四

### 常用参数

In [2]:
import json

data = {"name": "王五", "age": 30, "已婚": True, "子女": None}

# 美化输出（缩进2空格）
pretty = json.dumps(data, indent=2, ensure_ascii=False)
print(pretty)

# 排序键
sorted_json = json.dumps(data, sort_keys=True, ensure_ascii=False)
print(sorted_json)

# 过滤None值
filtered = json.dumps(data, skipkeys=False, ensure_ascii=False)
print(filtered)

{
  "name": "王五",
  "age": 30,
  "已婚": true,
  "子女": null
}
{"age": 30, "name": "王五", "子女": null, "已婚": true}
{"name": "王五", "age": 30, "已婚": true, "子女": null}


### 数据类型转换

In [3]:
import json

# Python → JSON 类型对应
# dict     → object
# list     → array
# str      → string
# int/float → number
# True     → true
# False    → false
# None     → null

data = {
    "string": "文本",
    "number": 42,
    "float": 3.14,
    "boolean": True,
    "none": None,
    "list": [1, 2, 3],
    "dict": {"key": "value"}
}

json_str = json.dumps(data, ensure_ascii=False, indent=2)
print(json_str)

{
  "string": "文本",
  "number": 42,
  "float": 3.14,
  "boolean": true,
  "none": null,
  "list": [
    1,
    2,
    3
  ],
  "dict": {
    "key": "value"
  }
}


### 解析大JSON文件

In [ ]:
import json
import ijson  # 需要安装: pip install ijson

def parse_large_json(filename):
    """流式解析大JSON文件"""
    with open(filename, "r", encoding="utf-8") as f:
        # 使用ijex解析，不一次性加载到内存
        parser = ijson.parse(f)
        for prefix, event, value in parser:
            if prefix.endswith(".name"):  # 只处理name字段
                print(f"找到名字: {value}")

# 或者使用json.loads逐行处理
def process_large_json_line_by_line(filename):
    """逐行处理JSON文件"""
    with open(filename, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():  # 跳过空行
                item = json.loads(line.strip())
                yield item

### JSON Path查询

In [ ]:
import json
from jsonpath_ng import jsonpath, parse  # 需要安装: pip install jsonpath-ng

data = {
    "store": {
        "book": [
            {"title": "Python入门", "price": 39.99},
            {"title": "数据分析", "price": 59.99},
            {"title": "机器学习", "price": 79.99}
        ]
    }
}

# 使用JSONPath查询
jsonpath_expr = parse("$.store.book[*].title")
matches = jsonpath_expr.find(data)

for match in matches:
    print(f"书名: {match.value}")  # Python入门, 数据分析, 机器学习

### JSON Schema验证

In [ ]:
import json
from jsonschema import validate, ValidationError  # 需要安装: pip install jsonschema

# 定义JSON Schema
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "minLength": 2},
        "age": {"type": "integer", "minimum": 0, "maximum": 150},
        "email": {"type": "string", "format": "email"}
    },
    "required": ["name", "age"]
}

# 验证数据
def validate_data(data_str):
    try:
        data = json.loads(data_str)
        validate(instance=data, schema=schema)
        return True, "验证通过"
    except ValidationError as e:
        return False, str(e)
    except json.JSONDecodeError as e:
        return False, f"JSON格式错误: {e}"

# 测试
test_data = '{"name": "张三", "age": 25, "email": "test@example.com"}'
print(validate_data(test_data))

## 注意事项

### 中文编码

In [6]:
import json

data = {"name": "张三", "city": "北京"}

# ❌ 默认（Unicode转义）
print(json.dumps(data))  
# {"name": "\u5f20\u4e09", "city": "\u5317\u4eac"}

# ✅ 保留中文
print(json.dumps(data, ensure_ascii=False))
# {"name": "张三", "city": "北京"}

{"name": "\u5f20\u4e09", "city": "\u5317\u4eac"}
{"name": "张三", "city": "北京"}


### 日期时间处理

In [7]:
import json
from datetime import datetime

data = {"time": datetime.now()}

# ❌ 直接序列化会报错
# print(json.dumps(data))

# ✅ 方法1：转换为字符串
data_str = {"time": datetime.now().isoformat()}
print(json.dumps(data_str))

# ✅ 方法2：自定义编码器
class DateTimeEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, datetime):
            return obj.isoformat()
        return super().default(obj)

print(json.dumps(data, cls=DateTimeEncoder))

{"time": "2025-12-17T11:30:39.050287"}
{"time": "2025-12-17T11:30:39.050287"}


### 循环引用处理

In [ ]:
import json

# 循环引用示例
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

a = Node(1)
b = Node(2)
a.next = b
b.next = a  # 循环引用

# ❌ 直接序列化会报错
# print(json.dumps(a.__dict__))

# ✅ 解决方法：打破循环引用
def serialize_node(node):
    visited = set()
    
    def _serialize(obj):
        if id(obj) in visited:
            return None  # 或返回其他标识
        visited.add(id(obj))
        
        if isinstance(obj, Node):
            return {
                "value": obj.value,
                "next": _serialize(obj.next) if obj.next else None
            }
        return obj
    
    return _serialize(node)

serialized = serialize_node(a)
print(json.dumps(serialized, indent=2))

## 实际案例

### API数据处理

In [ ]:
import json
import requests

def fetch_and_save(url, filename):
    """获取API数据并保存为JSON"""
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()  # 直接解析JSON
        
        # 保存到文件
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        
        # 提取需要的信息
        if "data" in data:
            return data["data"]
    return None

# 示例：获取GitHub用户信息
user_data = fetch_and_save("https://api.github.com/users/octocat", "user.json")

### 配置管理

In [ ]:
import json
import os

class Config:
    def __init__(self, filepath="config.json"):
        self.filepath = filepath
        self.config = self.load_config()
    
    def load_config(self):
        if os.path.exists(self.filepath):
            with open(self.filepath, "r", encoding="utf-8") as f:
                return json.load(f)
        return {"debug": False, "port": 8080, "host": "localhost"}
    
    def save_config(self):
        with open(self.filepath, "w", encoding="utf-8") as f:
            json.dump(self.config, f, indent=2, ensure_ascii=False)
    
    def get(self, key, default=None):
        return self.config.get(key, default)
    
    def set(self, key, value):
        self.config[key] = value
        self.save_config()

# 使用
config = Config()
print(config.get("port"))  # 8080
config.set("debug", True)